In [1]:
import fitz
path = "C:/DAEN690/2A13.pdf"
pdf = fitz.open(path)
pdf.metadata

{'format': 'PDF 1.6',
 'title': '2A13 TCDS',
 'author': 'DEBORAH JONES',
 'subject': '',
 'keywords': 'Manual',
 'creator': 'Microsoft® Word 2016',
 'producer': 'Adobe Acrobat Pro DC (32-bit) 21 Paper Capture Plug-in',
 'creationDate': "D:20211005120659-04'00'",
 'modDate': "D:20211005121358-04'00'",
 'trapped': '',
 'encryption': None}

In [2]:
# This Function is add the page number into get_text('word') 

In [3]:

def add_page(in_put, value):
    inn = []
    out = [[]]
    index = 0
#    v = str(value+1)
    for i in range(0, len(in_put)):
        if(i < len(in_put)-1):
            inn += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            inn.insert( 8, value+1)
            out[index] = inn
            index += 1
            inn = []
            out += [[]]
        else:
            inn += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            inn.insert( 8, value+1)
            out[index] = inn

    return out

In [4]:
# This function is to get all the [x0, y0, x1, y1, word, block_no, line_no, word_no] from the given TCDS file.

In [5]:
def get_all_text(data):
    pc = data.page_count
    word_box = []
    for i in range(0,pc):
        ct = data.load_page(i)
        gw = ct.get_text('words')
        # If the df do not need page anymore, #agw = ... and make wor_box += gw
        agw = add_page(gw, i)
        word_box += agw
    
    return word_box

In [6]:
# merge the words which have same block_no and line_no form all_text 

In [18]:
def word_line(in_put):
    index = 0
    word_list = []
    output = [[]]
    for i in range(0,len(in_put)-1):
        if(in_put[i][7]+1 == in_put[i+1][7]):
            word_list += [in_put[i][4]]  
        else:
            if(in_put[i+1][7]==0):
                word_list += [in_put[i][4]]
                word_list = [' '.join(word_list)]
                word_list += [in_put[i][5],in_put[i][6],in_put[i][7]]
                word_list.insert( 0, in_put[i-in_put[i][7]][0])
                word_list.insert( 1, in_put[i-in_put[i][7]][1])
                word_list.insert( 2, in_put[i][2])
                word_list.insert( 3, in_put[i][3])
            #print(word_list)

            output[index] += word_list
            index += 1
            word_list = []
            output += [[]]
    
    return output

In [8]:
# Get the X1 of first given word

In [26]:
def get_word_x1(df, word):
    
    for i in range(0,len(df)):
        if(df[i][4] == word):
            return df[i][2]

In [10]:
# define the line is [key_term, Model_header, information] to [Key term, header, info]

In [43]:
def detect_khi(in_put, x1):
    index = 0
    temp_list= []
    out_put = [[]]
    for i in range(0,len(in_put)-1):
        ## old one add & (in_put[i][6] == 0) in both if & elif
        if(in_put[i][0] <= x1) & (in_put[i][7] < 5 ):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'key_term')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        elif(in_put[i][0] <= x1) & (in_put[i][4].find('Approved') > 0):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'header')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        else:
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'info')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
    #print(out_put)
    return out_put

In [44]:
def all_process(pdf):
    all_text = get_all_text(pdf)
    all_line = word_line(all_text)
    out = word_line(all_text)
    x1 = get_word_x1(out, 'Fuel')
    kt = detect_khi(out, x1)


    print('----------------------------------------------Header-----------------------------------------------------')
    
    for i in range(0, len(kt)-1):
        if(kt[i][-1] == 'header'):
            print(kt[i][-5])
    print('-----------------------------------------------Key Term---------------------------------------------------') 
    
    for i in range(0, len(kt)-1):
        if(kt[i][-1] == 'key_term'):
            print(kt[i][-5])

In [45]:
all_process(pdf)

----------------------------------------------Header-----------------------------------------------------
I - Model PA-28-160 (Cherokee), 4 PCLM (Normal Category), Approved October 31, 1960, for S/N 28-03; 28-1
II - Model PA-28-150 (Cherokee), 4 PCLM (Normal Category), Approved June 2, 1961, for S/N 28-03; 28-1 through
III - Model PA-28-180 (Cherokee), 4 PCLM (Normal Category), Approved August 3, 1962; 2 PCLM (Utility Category),
IV - Model PA-28S-160 (Cherokee), 4 PCSM (Normal Category), Approved February 25, 1963, for S/N 28-1 through
V - Model PA-28S-180 (Cherokee), 4 PCSM (Normal Category), Approved May 10, 1963, for S/N 28-671 through 28-5859,
VI - Model PA-28-235 (Cherokee Pathfinder), 4 PCLM (Normal Category), Approved July 15, 1963, for S/N 28-10001
VIII - Model PA-28-140 (Cherokee Cruiser), 4 PCLM (Normal Category), 2 PCLM (Utility Category), Approved
IX - Model PA-28R-180 (Arrow), 4 PCLM (Normal Category), Approved June 8, 1967, for S/N 28R-30002 through
X - Model PA-28R-200 (